In [ ]:
import numpy as np
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator                                                                           
import os
import random
from pathlib import Path


In [ ]:
# Base directory containing numeric subdirectories
base_dir = Path(r'C:\Users\samir\Desktop\Zahraa\CMPS 261\Project\data')

# Mapping of indices to folder names (indices are the actual folder names)
categories = {
    1: "bottle",
    2: "basket",
    3: "food",
    4: "cup",
    5: "jar",
    6: "can",
    7: "dish",
    8: "mug",
    9: "glass"
}

# Initialize a dictionary to store file paths for each category
category_files = {index: [] for index in categories}

# Populate the dictionary with the file paths from the indexed folders
for index in categories:
    category_dir = base_dir / str(index)  # Numeric subdirectory name
    if category_dir.exists() and os.path.isdir(category_dir):
        # Collect all .jpg files under this directory
        category_files[index] = list(category_dir.rglob('*.jpg'))
    else:
        print(f"Warning: Directory not found or is not a directory: {category_dir}")


In [ ]:
train_files = []

for index, files in category_files.items():
    if files:
        train_files.extend(files)

# Print the number of files in the training set
print(f'Number of training files: {len(train_files)}')

In [ ]:
# Create DataFrames for training and testing sets
def create_dataframe(files, categories):
    data = []
    for file_path in files:
        index = int(Path(file_path).parent.name)
        label = categories.get(index, "Unknown")
        data.append({"Filepath": file_path, "Label": label})
    return pd.DataFrame(data)

train_df = create_dataframe(train_files, categories)

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    rotation_range=30,
    zoom_range=0.2,  # Increase zoom range
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],  # Adjust brightness
    fill_mode="nearest"
)


In [ ]:
train_df['Filepath'] = train_df['Filepath'].astype(str)

In [ ]:
# Confirm correct data generators for each dataset
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=0
)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the pre-trained MobileNetV2 model, excluding the top layer
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),  # Adjust input shape to match image shape
    include_top=False,
    weights='imagenet',
    pooling='avg'  # Pooling to obtain a feature vector
)

In [ ]:
# Freeze the pre-trained layers
pretrained_model.trainable = False
inputs = pretrained_model.input

x = pretrained_model.output  # Output of pre-trained model
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)

# Adjust the output layer to the number of classes in your dataset
outputs = tf.keras.layers.Dense(9, activation='softmax')(x)

# Create the final model by combining input and output layers
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train the model using the training and validation data generators
history = model.fit(
    train_images,
    epochs=10,  # Adjust epochs as needed
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            patience=2,
            restore_best_weights=True  # Restore weights of the best epoch based on loss
        )
    ]
)

# Save the weights of the best epoch based on training loss
best_weights = model.get_weights()
# Set the model's weights to the best weights obtained during training
model.set_weights(best_weights)

In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from pathlib import Path
import tensorflow as tf

# Load and preprocess all images in a directory
def load_and_preprocess_images(directory_path):
    # Initialize an empty list to store preprocessed images
    img_list = []
    
    # Iterate over each file in the directory
    for filename in os.listdir(directory_path):
        # Construct the full path to the image file
        img_path = os.path.join(directory_path, filename)
        
        # Load and resize the image
        img = image.load_img(img_path, target_size=(224, 224))
        
        # Convert the image to a numpy array
        img_array = image.img_to_array(img)
        
        # Preprocess the image for MobileNetV2
        img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
        
        # Append the processed image array to the list
        img_list.append(img_array)
    
    # Convert list of arrays to a single numpy array
    img_batch = np.array(img_list)
    return img_batch


In [ ]:
# Predict the classes of images
def predict_image_classes(model, img_batch):
    predictions = model.predict(img_batch)
    class_indices = np.argmax(predictions, axis=1)
    return class_indices